In [1]:
from typing import Iterator, Iterable, Callable

import spacy
from spacy import Errors
from spacy import displacy
from spacy.matcher import DependencyMatcher
from spacy.parts_of_speech import *
from spacy.tokens import Doc
from spacy.tokens import Span
from spacy.tokens import Token

from ttc.language.russian.constants import SPEAKING_VERBS

In [2]:
nlp = spacy.load("ru_core_news_sm")
nlp_md = spacy.load("ru_core_news_md")
# nlp_lg = spacy.load("ru_core_news_lg")

In [ ]:
doc = nlp("Один из них, худощавый, с подвижным лицом, приблизился к Гусеву и быстро проговорил.")
displacy.render(doc, style="dep")

In [ ]:
doc = nlp("– В Четвертом мосту грядут перемены, – объявил Каладин.")
displacy.render(doc, style="dep")

In [ ]:
doc = nlp("""Беги, парнишка, домой и передай мой привет своим близким!""")
displacy.render(doc.sents, style="dep")

In [ ]:
doc = nlp(
    "– Нет! –рявкнул Каладин. – Вылазки с мостом выматывают нас, потому что мы бóльшую часть времени бездельничаем. О, я знаю, нас заставляют работать – осматривать ущелья, чистить нужники, драить полы. Но солдаты не хотят, чтобы мы по-настоящему трудились, – им просто нужно нас занять. Поручая нам какое-нибудь дело, они про нас забывают. Поскольку я теперь ваш старшина, моя задача – сохранить вам жизнь. Стрелы паршенди никуда не исчезнут, поэтому я буду менять вас. Хочу сделать вас сильнее, чтобы на последнем отрезке пути с мостом, когда полетят стрелы, вы смогли бежать быстро. – Он посмотрел в глаза каждому. – Я собираюсь устроить так, чтобы Четвертый мост больше не потерял ни одного человека.")

displacy.render(doc, style="dep")

In [ ]:
doc = nlp("""Один Сафронов остался без сна. Он глядел на лежащих людей и с горечью высказывался:
— Эх ты, масса, масса. Трудно организовать из тебя скелет коммунизма! И что тебе надо? Стерве такой? Ты весь авангард, гадина, замучила!
И четко сознавая бедную отсталость масс, Сафронов прильнул к какому-то уставшему и забылся в глуши сна.
А утром он, не вставая с ложа, приветствовал девочку, пришедшую с Чиклиным, как элемент будущего и затем снова задремал.""")

displacy.render(doc, style="dep")

In [ ]:
doc = nlp("""Михайло важно снимает огромную Гришину ручищу с колен и отвечает:
— Ты меня не трожь.""")
displacy.render(doc, style="dep")

In [ ]:
doc = nlp_md("""Михайло важно снимает огромную Гришину ручищу с колен и
отвечает:
— Ты меня не трожь.""")
displacy.render(doc, style="dep")

In [3]:
from ttc.language.russian.constants import HYPHENS, CLOSE_QUOTES

from ttc.language.russian.constants import OPEN_QUOTES

Token.set_extension(
    "is_speaking_verb",
    force=True,
    getter=lambda t: any(sv in t.text for sv in SPEAKING_VERBS),
)

PREDICATIVE_TOKEN_EXTENSIONS = {
    "is_sent_end": lambda t: t.is_sent_end,
    "is_hyphen": lambda t: t.text in HYPHENS,
    "is_newline": lambda t: "\n" in t.text,
    "is_open_quote": lambda t: t.text in OPEN_QUOTES,
    "is_close_quote": lambda t: t.text in CLOSE_QUOTES,
    "is_speaking_verb": lambda t: any(v in t.lemma_ for v in SPEAKING_VERBS),
    "is_not_second_person": lambda t: "Person=Second" not in t.morph,
}

for name, pred in PREDICATIVE_TOKEN_EXTENSIONS.items():
    if not Token.has_extension(name):
        Token.set_extension(name, getter=pred)

In [ ]:
SPEAKER_TO_SPEAKING_VERB = [
    {  # (anchor) speaker
        "RIGHT_ID": "speaker",
        "RIGHT_ATTRS": {
            "DEP": "nsubj",
        },
    },
    {  # speaker <--- speaking verb
        "LEFT_ID": "speaker",
        "REL_OP": "<",
        "RIGHT_ID": "speaking_verb",
        "RIGHT_ATTRS": {
            "POS": "VERB",
            "_": {"is_speaking_verb": True},
        },
    },
]

SPEAKER_CONJUNCT_SPEAKING_VERB = [
    {  # (anchor) speaker
        "RIGHT_ID": "speaker",
        "RIGHT_ATTRS": {
            "DEP": "nsubj",
        },
    },
    {  # speaker <--- conjunct
        "LEFT_ID": "speaker",
        "REL_OP": "<",
        "RIGHT_ID": "conjunct",
        "RIGHT_ATTRS": {},
    },
    {  # conjunct ---> speaking verb
        "LEFT_ID": "conjunct",
        "REL_OP": ">",
        "RIGHT_ID": "speaking_verb",
        "RIGHT_ATTRS": {
            "POS": "VERB",
            "_": {"is_speaking_verb": True},
        },
    },
]

In [ ]:
text = """
— Что есть счастье? — вдруг громко спрашивает Гриша. Михайло смотрит на него, и вся физиономия его расплывается, как от сна. Всего полчаса назад он, отобрав четырех малышей шестилетнего возраста, лихо отплясывал с ними в хороводе, покуда не упал, чуть не раздавив одного из них.
Не получив ответа, Гриша жадно макает свою кудрявую голову в пиво, потом нагибается к Михайле, хлопает его по колену и хрипло говорит:
— Слышь, браток... Почему ты счастлив... Скажи... Корову подарю...
Михайло важно снимает огромную Гришину ручищу с колен и отвечает:
— Ты меня не трожь.
Гриша вздыхает.
— Ведь все вроде у меня есть, что у тебя... Корова, четыре бабы, хата с крышей, пчелы... Подумаю так: чево мне яще желать? Ничевошеньки. А автомобиля: «ЗИЛ» там или грузовик — мне и задаром не нужно: тише едешь, дальше будешь... Все у меня есть, — заключает Гриша.
Михайло молчит, утонув в пиве.
— Только мелочное все это, что у меня есть, — продолжает Гриша. — Не по размерам, а просто так, по душе... Мелочное, потому что мысли у меня есть. Оттого и страшно.
— Иди ты, — отвечает Михайло.
— Тоскливо мне чего-то жить, Мишук, — бормочет Гриша, опустив свою квадратную челюсть на стол.
— А чево?
— Да так... Тяжело все... Люди везде, комары... Опять же ночи... Облака... Очень скушно мне вставать по утрам... Руки... Сердце...
— Плохое это, — мычит Михайло.
Напившись пива, он становится разговорчивей, но так и не поднимая полностью завесы над своей великой тайной — тайной счастья. Лишь жирное, прыщеватое лицо его сияет как масленое солнышко.
— К бабе, к примеру, подход нужен, — поучает он, накрошив хлеба в рот. — Баба, она не корова, хоть и пузо у нее мягкое... Ее с замыслом выбирать нужно... К примеру, у меня есть девки на все случаи: одна, с которой я сплю завсегда после грозы, другая лунная (при луне, значит), с третьей — я только после баньки... Вот так.
Михайло совсем растаял от счастья и опять утонул в пиве.
— А меня все это не шевелит, — рассуждает Гриша. — Я и сам все это знаю.
— Счастье — это довольство... И чтоб никаких мыслей, — наконец проговаривается Михайло.
— Вот мыслей-то я и боюсь, — обрадовался Гриша. — Завсегда они у меня скачут. Удержу нет. И откуда только они появляются. Намедни совсем веселый был. Хотя и дочка кипятком обварилась. Шел себе просто по дороге, свистел. И увидал елочку, махонькую такую, облеванную... И так чего-то пужливо мне стало, пужливо... Или вот когда просто мысль появляется... Все ничего, ничего, пусто, и вдруг — бац! — мысль... Боязно очень. Особенно о себе боюсь думать.
— Ишь ты... О себе — оно иной раз бывает самое приятное думать, — скалится Михайло, поглаживая себя по животу.
В деревушке, как в лесу, не слышно ни единого непристойного звука. Все спит. Лишь вдали, поводя бедрами, выходит посмотреть на тучки упитанная дева, Тамарочка.
"""
doc_sm = nlp(text)
doc_md = nlp_md(text)

m = DependencyMatcher(nlp.vocab)
m.add("*", [SPEAKER_TO_SPEAKING_VERB])
m.add("**", [SPEAKER_CONJUNCT_SPEAKING_VERB])
x = list(doc_sm.sents)[8]
print(x, list(x)[7:11])
print(x[9], x[9].tag_, x[9].i, x[9].dep_, x[9].head, list(x[9].lefts), list
(x[9].rights), list(x[9].ancestors), list(x[9].children))
print(x[8], x[8].tag_, x[8].i, x[8].dep_)
print(x[2], x[2].tag_, x[2].i, x[2].dep_, x[2].head, list(x[2].lefts), list
(x[2].rights), list(x[2].ancestors), list(x[2].children))
print(x[1], x[1].tag_, x[1].i, x[1].dep_, x[1].head, list(x[1].lefts), list
(x[1].rights), list(x[1].ancestors), list(x[1].children))
print(x[0], x[0].tag_, x[0].i, x[0].dep_, x[0].head, list(x[0].lefts), list
(x[0].rights), list(x[0].ancestors), list(x[0].children))
print()
for match_id, token_ids in m(x):
    print([x[ti] for ti in token_ids])

displacy.render(doc_sm, style="dep")

print("\n")
m = DependencyMatcher(nlp_md.vocab)
m.add("*", [SPEAKER_TO_SPEAKING_VERB])
m.add("**", [SPEAKER_CONJUNCT_SPEAKING_VERB])
x = list(doc_md.sents)[8]
print(x, list(x)[7:11])
print(x[9], x[9].tag_, x[9].i, x[9].dep_, x[9].head, list(x[9].lefts), list
(x[9].rights), list(x[9].ancestors), list(x[9].children))
print(x[8], x[8].tag_, x[8].i, x[8].dep_)
print(x[2], x[2].tag_, x[2].i, x[2].dep_, x[2].head, list(x[2].lefts), list
(x[2].rights), list(x[2].ancestors), list(x[2].children))
print(x[1], x[1].tag_, x[1].i, x[1].dep_, x[1].head, list(x[1].lefts), list
(x[1].rights), list(x[1].ancestors), list(x[1].children))
print(x[0], x[0].tag_, x[0].i, x[0].dep_, x[0].head, list(x[0].lefts), list
(x[0].rights), list(x[0].ancestors), list(x[0].children))
print()
for match_id, token_ids in m(x):
    print([x[ti] for ti in token_ids])

displacy.render(doc_md, style="dep")

In [ ]:
doc = nlp_md("""Один из них, худощавый, с подвижным лицом, приблизился к Гусеву и быстро проговорил:
— Гусев Борис Владимирович. Вы арестованы.""")
displacy.render(doc, style="dep")

In [ ]:
def traverse_children(
    word: Token,
    children_selector: Callable[[Token], Iterable[Token]],
    dep_predicate: Callable[[Token], bool],
):
    children = list(children_selector(word))
    max_i = max((i for i, c in enumerate(children) if dep_predicate(c)),
                default=-1)
    return (
        traverse_children(
            children[max_i],
            children_selector,
            dep_predicate,
        )
        if max_i > -1
        else word
    )


def _noun_chunks(doclike: Doc | Span) -> Iterator[tuple[int, int, int]]:
    """Detect base noun phrases from a dependency parse. Works on Doc and Span."""
    doc = doclike.doc  # Ensure works on both Doc and Span.

    if not doc.has_annotation("DEP"):
        raise ValueError(Errors.E029)

    np_label = doc.vocab.strings.add("NP")
    labels = ["nsubj", "nsubj:pass", "obj", "iobj", "appos", "ROOT", "obl"]
    np_deps = [doc.vocab.strings.add(label) for label in labels]

    prev_end = -1

    for i, word in enumerate(doclike):
        # Prevent nested chunks from being produced
        if word.left_edge.i <= prev_end:
            continue

        if word.pos not in (NOUN, PROPN, PRON, NUM):
            continue

        if word.dep not in np_deps:
            continue

        start = traverse_children(
            word,
            lambda t: t.lefts,
            lambda t: t.dep_ in ("nmod", "amod", "obl", "flat:name", "appos"),
        )

        end = traverse_children(
            word,
            lambda t: t.rights,
            lambda t: (
                t.dep_ in (
                "nmod", "amod", "acl", "obl", "case", "flat:name", "appos")
                or t.dep_.startswith("acl")
            ),
        )

        prev_end = end.i
        yield start.i, end.i + 1, np_label

In [ ]:
doc = nlp("""Один из них, худощавый, с подвижным лицом, приблизился к Гусеву и быстро проговорил:
— Гусев Борис Владимирович. Вы арестованы.""")
ncs = [doc[a: b] for a, b, c in _noun_chunks(doc)]
print(" || ".join(str(s) for s in ncs))
print(doc.ents)
displacy.render(ncs[0], style="dep")

In [ ]:
doc = nlp("""Михайло молчит, утонув в пиве.
— Только мелочное все это, что у меня есть, — продолжает Гриша.""")
print([doc[a: b] for a, b, c in _noun_chunks(doc)])
print(doc.ents)
print(doc[doc[0].left_edge.i: doc[0].right_edge.i + 1])

In [ ]:
displacy.render(doc, style="dep")

In [ ]:
doc = nlp("Она позвонила мне из больницы, очень расстроенная.")
print([doc[a: b] for a, b, c in _noun_chunks(doc)])
displacy.render(doc, style="dep")

In [ ]:
doc = nlp(
    "— Я нашел твою девушку, — сказал Чиклин Прушевскому. — Пойдем смотреть ее, она еще цела.")
print([doc[a: b] for a, b, c in _noun_chunks(doc)])
displacy.render(doc, style="dep")

In [ ]:
doc = nlp_md("""Сзет потянулся за кружкой.
– Эй! – Тон быстро убирал ее. – Пиво-то не трожь! Я еще не допил!""")
print(list(doc.sents))
print([doc[a: b] for a, b, c in _noun_chunks(doc)])
displacy.render(doc, style="dep")

In [ ]:
doc = nlp("""– Если бы допил, – хмыкнул Тук, – ему бы нечего было вылить себе на голову, правда?
– Пусть он что-то другое сделает, – проворчал Тон.
– Ну ладно. – Тук вытащил засапожный нож.""")
print(list(doc.sents))
print([doc[a: b] for a, b, c in _noun_chunks(doc)])
displacy.render(doc, style="dep")

In [ ]:
doc = nlp("Да заткнись ты. — (")


def trim_span(span: Span, should_trim: Callable[[Token], bool]):
    doc = span.doc
    start = span.start
    while start < span.end and should_trim(doc[start]):
        start += 1
    end = span.end
    while end > span.start and should_trim(doc[end]):
        end -= 1
    return doc[start:end]


def should_trim(t: Token):
    return t.is_punct or t._.is_newline


trim_span(doc[0:-1], should_trim)